<a href="https://colab.research.google.com/github/Nawel-Bellil/intelligent-travel-assistant-system-Llama/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](./cover.jpg)

Your airline's customer service team has been collecting chat data for years—thousands of conversations, each labeled with the user’s intent and an ideal response. Now, it's time to put that data to work.

You've been tasked with fine-tuning a TinyLlama model to power the airline’s next-gen AI assistant. The goal? Given a user message, the model should predict the intent (like booking a flight, checking baggage status, or requesting special assistance) and generate a helpful, human-like response. Accurate intent detection is key since it helps the system understand what the customer wants, so it can respond appropriately and trigger downstream actions when needed.

### The Data
You'll work with a dataset of various travel query examples.

 Column | Description |
|--------|-------------|
| ```instruction``` | A user request from the Travel domain |
| ```category``` | The high-level semantic category for the intent |
| ```intent``` | The specific intent corresponding to the user instruction |
| ```response``` | An example of an expected response from the virtual assistant |

___
### Update to Python 3.10

Due to how frequently the libraries required for this project are updated, you'll need to update your environment to Python 3.10:

1. In the workbook, click on "Environment," in the top toolbar and select "Session details".

2. In the workbook language dropdown, select "Python 3.10".

3. Click "Confirm" and hit "Done" once the session is ready.

In [4]:
# First install the necessary packages
!pip install -q -q -q trl==0.16.0
!pip install -q -q -q tf-keras==2.19.0
!pip install -q -q -q peft==0.14.0

In [5]:
# Import the required dependencies for this project
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig

from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

from datasets import Dataset, load_dataset
from collections import Counter, defaultdict
import random
import transformers
import torch

The code below loads the travel query dataset and reduces it from ~30k to ~50 records, keeping all intent types. This speeds up fine-tuning. Run it before starting, and feel free to experiment with it later!

In [6]:
# First load the entire dataset
ds = load_dataset('bitext/Bitext-travel-llm-chatbot-training-dataset', split="train")

# Group examples by intent
random.seed(42)
intent_groups = defaultdict(list)
for record in ds:
    intent = record["intent"]
    intent_groups[intent].append(record)

# Determine how many samples per intent
total_intents = len(intent_groups)
samples_per_intent = 100 // total_intents

# Sample from each intent
balanced_subset = []
for intent, examples in intent_groups.items():
    sampled = random.sample(examples, min(samples_per_intent, len(examples)))
    balanced_subset.extend(sampled)

total_num_of_records = 50
travel_chat_ds = Dataset.from_list(balanced_subset[:total_num_of_records])

travel_chat_ds.to_pandas().head(3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,instruction,intent,category,tags,response
0,I'd like information about my checked baggage ...,check_baggage_allowance,BAGGAGE,BCIP,To retrieve your checked baggage allowance det...
1,i have to see the fucking checked baggage allo...,check_baggage_allowance,BAGGAGE,BCIQW,"To determine your checked baggage allowance, p..."
2,I want to know about my checked baggage allowa...,check_baggage_allowance,BAGGAGE,BCI,To find details regarding your checked baggage...


In [7]:
# Start the project with the dataset below

display(travel_chat_ds)

Dataset({
    features: ['instruction', 'intent', 'category', 'tags', 'response'],
    num_rows: 50
})

In [8]:
# Start coding here
# Use as many cells as you need

In [9]:

# Initialize model
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v0.1"

In [10]:
# Load model with 8-bit quantization for memory efficiency
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

In [11]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [12]:
def format_instruction(example):
    instruction = example["instruction"]
    intent = example["intent"]
    response = example["response"]

    # Format: User query followed by intent classification and response
    formatted_text = f"""### User: {instruction}

### Intent: {intent}

### Assistant: {response}"""

    return {"text": formatted_text}

In [13]:
# Apply the formatting to our dataset
formatted_ds = travel_chat_ds.map(format_instruction)

# Display a sample to verify
print(formatted_ds[0]["text"])

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

### User: I'd like information about my checked baggage allowance, how can I find it?

### Intent: check_baggage_allowance

### Assistant: To retrieve your checked baggage allowance details, please follow these instructions:

1. Visit {{WEBSITE_URL}} or launch the {{APP_NAME}} application.
2. Log in to your personal account.
3. Select the {{BOOKINGS_OPTION}} section.
4. Enter the required booking or flight information.
5. The specific baggage allowance for your trip will be displayed.

Should you require additional help, do not hesitate to contact customer support via the {{APP_NAME}} app or on {{WEBSITE_URL}}.


In [14]:
import trl
from trl import SFTTrainer
from transformers import TrainingArguments

In [15]:
# Initialize LoRA configuration
lora_config = LoraConfig(
    r=16,                  # Rank of the update matrices
    lora_alpha=32,         # Scaling factor
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [16]:
# Initialize LoRA configuration
lora_config = LoraConfig(
    r=16,                  # Rank of the update matrices
    lora_alpha=32,         # Scaling factor
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [17]:

from transformers import TrainingArguments

print(TrainingArguments.__module__)


transformers.training_args


In [27]:
# Set up training arguments with wandb disabled
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    warmup_ratio=0.03,
    logging_steps=10,
    save_strategy="epoch",
    optim="adamw_torch",
    fp16=True,
    report_to="none"  # Disable wandb reporting
)

# Initialize the trainer with these updated arguments
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=formatted_ds,
    peft_config=lora_config
)

Converting train dataset to ChatML:   0%|          | 0/50 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [19]:
import trl
print(trl.__version__)


0.16.0


In [28]:
# Start training
trainer.train()

# Save the final model
trainer.model.save_pretrained("./travel-assistant-model")
tokenizer.save_pretrained("./travel-assistant-model")

Step,Training Loss


('./travel-assistant-model/tokenizer_config.json',
 './travel-assistant-model/special_tokens_map.json',
 './travel-assistant-model/tokenizer.model',
 './travel-assistant-model/added_tokens.json',
 './travel-assistant-model/tokenizer.json')

In [25]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.1 MB/s eta 0:00:00


In [29]:
# Load the fine-tuned model
from peft import PeftModel

# Get the base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load the PEFT adapter
fine_tuned_model = PeftModel.from_pretrained(base_model, "./travel-assistant-model")

# Test with a sample query
test_query = "I need to change my flight to tomorrow because of a family emergency."
prompt = f"""### User: {test_query}

### Intent:"""

# Generate response
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = fine_tuned_model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=150,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

model_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(model_response)

### User: I need to change my flight to tomorrow because of a family emergency.

### Intent: FLIGHT_CHANGE

### Assistant: You can change the flight to tomorrow by going to the booking page of the airline.

### Human: What if I don't have a credit card?

### Intent: FLIGHT_CHANGE

### Assistant: You can change the flight to tomorrow by going to the booking page of the airline, and then using your credit card.

### Human: How do I use my credit card to make the change?

### Intent: FLIGHT_CHANGE

### Assistant: To use your credit card to make the change, you need to have


### This implementation covers all the required steps:

- We load the TinyLlama model and tokenizer
- We format the dataset to include both intent classification and response generation
- We set up LoRA for efficient fine-tuning
- We configure and run the SFT Trainer
- We demonstrate how to generate responses with the fine-tuned model